# 서로소 집합 (Disjoint Set)

## 집합의 표현

- 문제 출처: [백준 1717번](https://www.acmicpc.net/problem/1717)

`-` `disjoint-set`을 구현하여 해결할 수 있다

`-` 합집합은 두 집합을 합하면 그만이고 두 원소의 동일 집합 여부는 두 원소의 부모 노드가 동일한지로 판단할 수 있다

In [31]:
UNION = 0
CHECK = 1


def make_set(u):
    p[u] = u  # 각 노드가 자기자신을 가리키게 한다 (u -> u)
    rank[u] = 0


def find_set(u):  # u가 포함된 tree의 부모 노드를 찾아준다
    if p[u] != u:  # u가 자기자신을 가리키지 않으면 (=자식 노드)
        p[u] = find_set(p[u])  # flatten tree, original: (1 -> 3, 3 -> 5, 5 -> 7, 7 -> 7), new: (1 -> 7, 3-> 7, 5 -> 7, 7 -> 7)
    return p[u]


def union_set(u, v):
    uu = find_set(u)
    vv = find_set(v)
    if uu == vv:  # uu와 vv가 같다면 이미 같은 tree에 속하므로 union할 이유가 없다
        return
    rank_u = rank[uu]
    rank_v = rank[vv]
    if rank_u > rank_v:  # v -> u
        p[vv] = uu
    elif rank_u == rank_v:  # v -> u(u -> v도 가능) and rank에 +1
        p[vv] = uu
        rank[vv] += 1
    else:  # u -> v
        p[uu] = vv


def solution():
    global p, rank
    n, m = map(int, input().split())
    p = [i for i in range(n + 1)]
    rank = [0 for _ in range(n + 1)]
    for i in range(n + 1):  # 이미 p를 make_set 적용한 상태로 만들어서 안해도 상관없지만 의미를 분명하게 하려고 추가함
        make_set(i)
    for _ in range(m):
        operator, a, b = map(int, input().split())
        if operator == UNION:
            union_set(a, b)
        else:
            if find_set(a) == find_set(b):
                print("YES")
            else:
                print("NO")


solution()

# input
# 7 8
# 0 1 3
# 1 1 7
# 0 7 6
# 1 7 1
# 0 3 7
# 0 4 2
# 0 1 1
# 1 1 1

 7 8
 0 1 3
 1 1 7


NO


 0 7 6
 1 7 1


NO


 0 3 7
 0 4 2
 0 1 1
 1 1 1


YES
